# Importing Libraries

In [1]:
import os
import sys
import time
import torch
import random
import network
import argparse
import platform
import ivtmetrics # You must "pip install ivtmetrics" to use
import dataloader
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from skimage.metrics import structural_similarity as ssim
import numpy as np
# import lpips
import piq
from torchvision.utils import save_image
from torchvision.utils import save_image
import torchvision.transforms as transforms
from PIL import Image

# Argument Parser

## In case of running using cmd

In [2]:
#%% @args parsing
parser = argparse.ArgumentParser()

parser.add_argument('--model', type=str, default='rendezvous', choices=['rendezvous'], help='Model name?')
parser.add_argument('--version', type=int, default=0,  help='Model version control (for keeping several versions)') 
parser.add_argument('--hr_output', action='store_true', help='Whether to use higher resolution output (32x56) or not (8x14). Default: False')
parser.add_argument('--use_ln', action='store_true', help='Whether to use layer norm or batch norm in AddNorm() function. Default: False')
parser.add_argument('--decoder_layer', type=int, default=8, help='Number of MHMA layers ') 
# job
parser.add_argument('-t', '--train', action='store_true', help='to train.')
parser.add_argument('-e', '--test',  action='store_true', help='to test')
parser.add_argument('--val_interval', type=int, default=1,  help='(for hp tuning). Epoch interval to evaluate on validation data. set -1 for only after final epoch, or a number higher than the total epochs to not validate.')
# data
parser.add_argument('--data_dir', type=str, default='/path/to/dataset', help='path to dataset?')
parser.add_argument('--dataset_variant', type=str, default='cholect45-crossval', choices=['cholect50', 'cholect45', 'cholect50-challenge', 'cholect50-crossval', 'cholect45-crossval'], help='Variant of the dataset to use')
parser.add_argument('-k', '--kfold', type=int, default=1,  choices=[1,2,3,4,5,], help='The test split in k-fold cross-validation')
parser.add_argument('--image_width', type=int, default=448, help='Image width ')  
parser.add_argument('--image_height', type=int, default=256, help='Image height ')  
parser.add_argument('--image_channel', type=int, default=3, help='Image channels ')  
parser.add_argument('--num_tool_classes', type=int, default=6, help='Number of tool categories')
parser.add_argument('--num_verb_classes', type=int, default=10, help='Number of verb categories')
parser.add_argument('--num_target_classes', type=int, default=15, help='Number of target categories')
parser.add_argument('--num_triplet_classes', type=int, default=100, help='Number of triplet categories')
parser.add_argument('--augmentation_list', type=str, nargs='*', default=['original', 'vflip', 'hflip', 'contrast', 'rot90'], help='List augumentation styles (see dataloader.py for list of supported styles).')
# hp
parser.add_argument('-b', '--batch', type=int, default=32,  help='The size of sample training batch')
parser.add_argument('--epochs', type=int, default=100,  help='How many training epochs?')
parser.add_argument('-w', '--warmups', type=int, nargs='+', default=[9,18,58], help='List warmup epochs for tool, verb-target, triplet respectively')
parser.add_argument('-l', '--initial_learning_rates', type=float, nargs='+', default=[0.01, 0.01, 0.01], help='List learning rates for tool, verb-target, triplet respectively')
parser.add_argument('--weight_decay', type=float, default=1e-5,  help='L2 regularization weight decay constant')
parser.add_argument('--decay_steps', type=int, default=10,  help='Step to exponentially decay')
parser.add_argument('--decay_rate', type=float, default=0.99,  help='Learning rates weight decay rate')
parser.add_argument('--momentum', type=float, default=0.95,  help="Optimizer's momentum")
parser.add_argument('--power', type=float, default=0.1,  help='Learning rates weight decay power')
# weights
parser.add_argument('--pretrain_dir', type=str, default='', help='path to pretrain_weight?')
parser.add_argument('--test_ckpt', type=str, default=None, help='path to model weight for testing')
# device
parser.add_argument('--gpu', type=str, default="0",  help='The gpu device to use. To use multiple gpu put all the device ids comma-separated, e.g: "0,1,2" ')
FLAGS, unparsed = parser.parse_known_args()
import torch

# Parameters Definition

In [3]:
#%% @params definitions
is_train        = FLAGS.train
is_test         = FLAGS.test
dataset_variant = FLAGS.dataset_variant
data_dir        = FLAGS.data_dir
kfold           = FLAGS.kfold if "crossval" in dataset_variant else 0
version         = FLAGS.version
hr_output       = FLAGS.hr_output
use_ln          = FLAGS.use_ln
batch_size      = FLAGS.batch
pretrain_dir    = FLAGS.pretrain_dir
test_ckpt       = FLAGS.test_ckpt
weight_decay    = FLAGS.weight_decay
learning_rates  = FLAGS.initial_learning_rates
warmups         = FLAGS.warmups
decay_steps     = FLAGS.decay_steps
decay_rate      = FLAGS.decay_rate
power           = FLAGS.power
momentum        = FLAGS.momentum
epochs          = FLAGS.epochs
gpu             = FLAGS.gpu
image_height    = FLAGS.image_height
image_width     = FLAGS.image_width
image_channel   = FLAGS.image_channel
num_triplet     = FLAGS.num_triplet_classes
num_tool        = FLAGS.num_tool_classes
num_verb        = FLAGS.num_verb_classes
num_target      = FLAGS.num_target_classes
val_interval    = FLAGS.epochs-1 if FLAGS.val_interval==-1 else FLAGS.val_interval
set_chlg_eval   = True if "challenge" in dataset_variant else False # To observe challenge evaluation protocol
gpu             = ",".join(str(FLAGS.gpu).split(","))
decodelayer     = FLAGS.decoder_layer
addnorm         = "layer" if use_ln else "batch"
modelsize       = "high" if hr_output else "low"
FLAGS.multigpu  = len(gpu) > 1  # not yet implemented !
mheaders        = ["","l", "cholect", "k"]
margs           = [FLAGS.model, decodelayer, dataset_variant, kfold]
wheaders        = ["norm", "res"]
wargs           = [addnorm, modelsize]
modelname       = "_".join(["{}{}".format(x,y) for x,y in zip(mheaders, margs) if len(str(y))])+"_"+\
                  "_".join(["{}{}".format(x,y) for x,y in zip(wargs, wheaders) if len(str(x))])
model_dir       = "./__checkpoint__/run_{}".format(version)

if not os.path.exists(model_dir): os.makedirs(model_dir)
resume_ckpt     = None
ckpt_path       = os.path.join(model_dir, '{}.pth'.format(modelname))
logfile         = os.path.join(model_dir, '{}.log'.format(modelname))
data_augmentations      = FLAGS.augmentation_list 
iterable_augmentations  = []
print("Configuring network ...")

#%% @functions (helpers)
def assign_gpu(gpu=None):  
    os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu) 
    os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1' 
    

def get_weight_balancing(case='cholect50'):
    # 50:   cholecT50, data splits as used in rendezvous paper
    # 50ch: cholecT50, data splits as used in CholecTriplet challenge
    # 45cv: cholecT45, official data splits (cross-val)
    # 50cv: cholecT50, official data splits (cross-val)
    switcher = {
        'cholect50': {
            'tool'  :   [0.08084519, 0.81435289, 0.10459284, 2.55976864, 1.630372490, 1.29528455],
            'verb'  :   [0.31956735, 0.07252306, 0.08111481, 0.81137309, 1.302895320, 2.12264151, 1.54109589, 8.86363636, 12.13692946, 0.40462028],
            'target':   [0.06246232, 1.00000000, 0.34266478, 0.84750219, 14.80102041, 8.73795181, 1.52845100, 5.74455446, 0.285756500, 12.72368421, 0.6250808,  3.85771277, 6.95683453, 0.84923888, 0.40130032]
        },
        'cholect50-challenge': {
            'tool':     [0.08495163, 0.88782288, 0.11259564, 2.61948830, 1.784866470, 1.144624170],
            'verb':     [0.39862805, 0.06981640, 0.08332925, 0.81876204, 1.415868390, 2.269359150, 1.28428410, 7.35822511, 18.67857143, 0.45704490],
            'target':   [0.07333818, 0.87139287, 0.42853950, 1.00000000, 17.67281106, 13.94545455, 1.44880997, 6.04889590, 0.326188650, 16.82017544, 0.63577586, 6.79964539, 6.19547658, 0.96284208, 0.51559559]
        },
        'cholect45-crossval': {
            1: {
                'tool':     [0.08165644, 0.91226868, 0.10674758, 2.85418156, 1.60554885, 1.10640067],
                'verb':     [0.37870137, 0.06836869, 0.07931255, 0.84780024, 1.21880342, 2.52836879, 1.30765704, 6.88888889, 17.07784431, 0.45241117],
                'target':   [0.07149629, 1.0, 0.41013597, 0.90458015, 13.06299213, 12.06545455, 1.5213205, 5.04255319, 0.35808332, 45.45205479, 0.67493897, 7.04458599, 9.14049587, 0.97330595, 0.52633249]
                },
            2: {
                'tool':     [0.0854156, 0.89535362, 0.10995253, 2.74936869, 1.78264429, 1.13234529],
                'verb':     [0.36346863, 0.06771776, 0.07893261, 0.82842725, 1.33892161, 2.13049748, 1.26120359, 5.72674419, 19.7, 0.43189126],
                'target':   [0.07530655, 0.97961957, 0.4325135, 0.99393438, 15.5387931, 14.5951417, 1.53862569, 6.01836394, 0.35184462, 15.81140351, 0.709506, 5.79581994, 8.08295964, 1.0, 0.52689272]
            },
            3: {
                "tool" :   [0.0915228, 0.89714969, 0.12057004, 2.72128174, 1.94092281, 1.12948557],
                "verb" :   [0.43636862, 0.07558554, 0.0891017, 0.81820519, 1.53645582, 2.31924198, 1.28565657, 6.49387755, 18.28735632, 0.48676763],
                "target" : [0.06841828, 0.90980736, 0.38826607, 1.0, 14.3640553, 12.9875, 1.25939394, 5.38341969, 0.29060227, 13.67105263, 0.59168565, 6.58985201, 5.72977941, 0.86824513, 0.47682423]

            },
            4: {
                'tool':     [0.08222218, 0.85414117, 0.10948695, 2.50868784, 1.63235867, 1.20593318],
                'verb':     [0.41154261, 0.0692142, 0.08427214, 0.79895288, 1.33625219, 2.2624166, 1.35343681, 7.63, 17.84795322, 0.43970609],
                'target':   [0.07536126, 0.85398445, 0.4085784, 0.95464422, 15.90497738, 18.5978836, 1.55875831, 5.52672956, 0.33700863, 15.41666667, 0.74755423, 5.4921875, 6.11304348, 1.0, 0.50641118],
            },
            5: {
                'tool':     [0.0804654, 0.92271157, 0.10489631, 2.52302243, 1.60074906, 1.09141982],
                'verb':     [0.50710436, 0.06590258, 0.07981184, 0.81538866, 1.29267277, 2.20525568, 1.29699248, 7.32311321, 25.45081967, 0.46733895],
                'target':   [0.07119395, 0.87450495, 0.43043372, 0.86465981, 14.01984127, 23.7114094, 1.47577277, 5.81085526, 0.32129865, 22.79354839, 0.63304067, 6.92745098, 5.88833333, 1.0, 0.53175798]
            }
        },
        'cholect50-crossval': {
            1:{
                'tool':     [0.0828851, 0.8876, 0.10830995, 2.93907285, 1.63884786, 1.14499484],
                'verb':     [0.29628942, 0.07366916, 0.08267971, 0.83155428, 1.25402434, 2.38358209, 1.34938741, 7.56872038, 12.98373984, 0.41502079],
                'target':   [0.06551745, 1.0, 0.36345711, 0.82434783, 13.06299213, 8.61818182, 1.4017744, 4.62116992, 0.32822238, 45.45205479, 0.67343211, 4.13200498, 8.23325062, 0.88527215, 0.43113306],

            },
            2:{
                'tool':     [0.08586283, 0.87716737, 0.11068887, 2.84210526, 1.81016949, 1.16283571],
                'verb':     [0.30072757, 0.07275414, 0.08350168, 0.80694143, 1.39209979, 2.22754491, 1.31448763, 6.38931298, 13.89211618, 0.39397505],
                'target':   [0.07056703, 1.0, 0.39451115, 0.91977006, 15.86206897, 9.68421053, 1.44483706, 5.44378698, 0.31858714, 16.14035088, 0.7238395, 4.20571429, 7.98264642, 0.91360477, 0.43304307],
            },
            3:{
            'tool':      [0.09225068, 0.87856006, 0.12195811, 2.82669323, 1.97710987, 1.1603972],
                'verb':     [0.34285159, 0.08049804, 0.0928239, 0.80685714, 1.56125608, 2.23984772, 1.31471136, 7.08835341, 12.17241379, 0.43180428],
                'target':   [0.06919395, 1.0, 0.37532866, 0.9830703, 15.78801843, 8.99212598, 1.27597765, 5.36990596, 0.29177312, 15.02631579, 0.64935557, 5.08308605, 5.86643836, 0.86580743, 0.41908257], 
            },
            4:{
                'tool':     [0.08247885, 0.83095539, 0.11050268, 2.58193042, 1.64497676, 1.25538881],
                'verb':     [0.31890981, 0.07380354, 0.08804592, 0.79094077, 1.35928144, 2.17017208, 1.42947103, 8.34558824, 13.19767442, 0.40666428],
                'target':   [0.07777646, 0.95894072, 0.41993829, 0.95592153, 17.85972851, 12.49050633, 1.65701092, 5.74526929, 0.33763901, 17.31140351, 0.83747083, 3.95490982, 6.57833333, 1.0, 0.47139615],
            },
            5:{
                'tool':     [0.07891691, 0.89878025, 0.10267677, 2.53805556, 1.60636428, 1.12691169],
                'verb':     [0.36420961, 0.06825313, 0.08060635, 0.80956984, 1.30757221, 2.09375, 1.33625848, 7.9009434, 14.1350211, 0.41429631],
                'target':   [0.07300329, 0.97128713, 0.42084942, 0.8829883, 15.57142857, 19.42574257, 1.56521739, 5.86547085, 0.32732733, 25.31612903, 0.70171674, 4.55220418, 6.13125, 1.0, 0.48528321],
            }
        }
    }
    return switcher.get(case)
     

Configuring network ...


# Loading and Building Model

In [4]:
# Path to model checkpoint
test_ckpt = 'weights/rendezvous_l8_cholectcholect45-crossval_k1_batchnorm_lowres_180.pth'

# Load base structure model
model = network.Rendezvous('resnet18', hr_output=hr_output, use_ln=use_ln).cuda()
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_train_params = sum(p.numel() for p in model.parameters() if p.requires_grad)


#%% performance tracker for hp tuning
benchmark   = torch.nn.Parameter(torch.tensor([0.0]), requires_grad=False)
print("Model built ...")

# Load the checkpoint of the trained model
if os.path.exists(test_ckpt):
    model.load_state_dict(torch.load(test_ckpt) ,  strict=False)

/home/umairnawaz/.conda/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/umairnawaz/.conda/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model built ...


# Define Loss, Activation, and mAP Metrics

In [5]:
# Or constant weights from average of the random sampling of the dataset: we found this to produce better result.
tool_weight     = [0.93487068, 0.94234964, 0.93487068, 1.18448115, 1.02368339, 0.97974447]
verb_weight     = [0.60002400, 0.60002400, 0.60002400, 0.61682467, 0.67082683, 0.80163207, 0.70562823, 2.11208448, 2.69230769, 0.60062402]
target_weight   = [0.49752894, 0.52041527, 0.49752894, 0.51394739, 2.71899565, 1.75577963, 0.58509403, 1.25228034, 0.49752894, 2.42993134, 0.49802647, 0.87266576, 1.36074165, 0.50150917, 0.49802647]


#%% Loss
activation  = nn.Sigmoid()
loss_fn_i   = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(tool_weight).cuda())
loss_fn_v   = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(verb_weight).cuda())
loss_fn_t   = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(target_weight).cuda())
loss_fn_ivt = nn.BCEWithLogitsLoss()

#Monkey patches
np.float = float    
np.int = int
np.object = object
np.bool = bool  

#%% evaluation metrics
mAP = ivtmetrics.Recognition(100)
mAP.reset_global()
if not set_chlg_eval:
    mAPi = ivtmetrics.Recognition(6)
    mAPv = ivtmetrics.Recognition(10)
    mAPt = ivtmetrics.Recognition(15)
    mAPi.reset_global()
    mAPv.reset_global()
    mAPt.reset_global()

#%% Adversarial metrics
mAP_adv = ivtmetrics.Recognition(100)
mAP_adv.reset_global()
if not set_chlg_eval:
    mAPi_adv = ivtmetrics.Recognition(6)
    mAPv_adv = ivtmetrics.Recognition(10)
    mAPt_adv = ivtmetrics.Recognition(15)
    mAPi_adv.reset_global()
    mAPv_adv.reset_global()
    mAPt_adv.reset_global()
print("Metrics built ...")

Metrics built ...


# Dataset Loading

In [6]:
#%% data loading : variant and split selection (Note: original paper used different augumentation per epoch)
data_dir = '/share/sdb/umairnawaz/Data/'
dataset_variant= 'cholect45-crossval'
dataset = dataloader.CholecT50( 
            dataset_dir=data_dir, 
            dataset_variant=dataset_variant,
            test_fold=kfold,
            augmentation_list=data_augmentations,
            )

# build dataset
train_dataset, val_dataset, test_dataset = dataset.build()

# Load only test dataset in dataloader

In [7]:
test_dataloaders = []
for video_dataset in test_dataset:
    test_dataloader = DataLoader(video_dataset, batch_size=batch_size, shuffle=False, prefetch_factor=3*batch_size, num_workers=3, pin_memory=True, persistent_workers=True, drop_last=False)
    test_dataloaders.append(test_dataloader)
print("Dataset loaded ...")

Dataset loaded ...


In [8]:
# Device to load the model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to perform FGSM attack
def fast_gradient_sign_method(model, x, y, eps=8/255):    

    # Original Images
    x = x.clone().detach().cuda().requires_grad_(True)
    
    targeted = False  # Change to True if we have a targeted attack
    y_target = None    # Set the target label if it's a targeted attack

    # Forward pass
    tool, verb, target, triplet = model(x)

    # Extracting Logits and CAMs of each category
    
    cam_i, logit_i  = tool
    cam_v, logit_v  = verb
    cam_t, logit_t  = target
    logit_ivt       = triplet            

    # Compute Loss
    loss_i          = loss_fn_i(logit_i, y[0].float())
    loss_v          = loss_fn_v(logit_v, y[1].float())
    loss_t          = loss_fn_t(logit_t, y[2].float())
    loss_ivt        = loss_fn_ivt(logit_ivt, y[3].float()) 

    # Total Loss
    loss = (loss_i) + (loss_v) + (loss_t) + loss_ivt 

    # Find the gradient
    grad = torch.autograd.grad(
        loss, x, retain_graph=False, create_graph=False, allow_unused=True
    )[0]

    # Update adversarial images
    if grad is not None:
        # print("Varifying Gradient")
        x_adv = x + (eps * grad.sign())
        x_adv = torch.clamp(x_adv, min=0, max=1).detach()

    return x_adv

## Spatial Smoothing

In [9]:
import logging
from typing import Optional, Tuple

import numpy as np
from scipy.ndimage import median_filter

from art.utils import CLIP_VALUES_TYPE
from art.defences.preprocessor.preprocessor import Preprocessor

/home/umairnawaz/.conda/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
params = ["window_size", "channels_first", "clip_values"]

def spatialSmoothing(x, y=None):
    window_size = 3  # Fixed: Removed the comma to define it as an int, not a tuple
    channels_first = False
    clip_values = None

    x_ndim = x.ndim
    # print("The dimensions are: ", x_ndim)
    if x_ndim not in [4, 5]:
        raise ValueError("Unrecognized input dimension. Spatial smoothing can only be applied to image and video data.")

    # get channel index
    channel_index = 1 if channels_first else x_ndim - 1

    filter_size = [window_size] * (x_ndim - 2)  # Adjusted for the spatial dimensions only
    filter_size = [1] + filter_size + [1]  # Adjust for batch and channel dimensions
    print("Filter size: ", filter_size)

    # print("Original tensor shape: ", x.shape)
    if x.is_cuda:
        x = x.cpu()  # Ensure tensor is on CPU
    x = x.permute(0, 2, 3, 1).numpy()  # Rearrange and convert to NumPy
    # print("After permute and conversion shape: ", x.shape)
    
    result = median_filter(x, size=tuple(filter_size), mode="reflect")

    if clip_values is not None:
        np.clip(result, clip_values[0], clip_values[1], out=result)

    
    result_tensor = torch.from_numpy(result)
    result_tensor = result_tensor.permute(0, 3, 1, 2)
    # Move the tensor to GPU if your original tensor was on a GPU
    if torch.cuda.is_available():
        result_tensor = result_tensor.to('cuda')

    return result_tensor, y

def _check_params(self) -> None:
    if not (isinstance(self.window_size, int) and self.window_size > 0):
        raise ValueError("Sliding window size must be a positive integer.")

    if self.clip_values is not None and len(self.clip_values) != 2:
        raise ValueError("'clip_values' should be a tuple of 2 floats or arrays containing the allowed data range.")

    if self.clip_values is not None and np.array(self.clip_values[0] >= self.clip_values[1]).any():
        raise ValueError("Invalid 'clip_values': min >= max.")

## Pixel Defend

In [21]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging
from typing import Optional, Tuple, TYPE_CHECKING

import numpy as np
from tqdm.auto import tqdm

from art.config import ART_NUMPY_DTYPE
from art.defences.preprocessor.preprocessor import Preprocessor

if TYPE_CHECKING:
    from art.utils import CLIP_VALUES_TYPE, CLASSIFIER_NEURALNETWORK_TYPE


In [22]:
def pixelDefend(x, y = None):
    clip_values = (0.0, 1.0),
    eps= 16,
    pixel_cnn = None,
    batch_size = 128,
    apply_fit = False,
    apply_predict = True,
    verbose = False,
    """
    Apply pixel defence to sample `x`.

    :param x: Sample to defense with shape `(batch_size, width, height, depth)`. `x` values are expected to be in
            the data range [0, 1].
    :param y: Labels of the sample `x`. This function does not affect them in any way.
    :return: Purified sample.
    """
    # Convert into `uint8`
    original_shape = x.shape
    if pixel_cnn is not None:
        activations = pixel_cnn.get_activations(x, layer=-1, batch_size=batch_size)
        if isinstance(activations, np.ndarray):
            probs = activations.reshape((x.shape[0], -1, 256))
        else:
            raise ValueError("Activations are None.")
    else:
        raise ValueError("No model received for `pixel_cnn`.")

    x = x * 255
    x = x.astype("uint8")
    x = x.reshape((x.shape[0], -1))

    # Start defence one image at a time
    for i, x_i in enumerate(tqdm(x, desc="PixelDefend", disable=not verbose)):
        for feat_index in range(x.shape[1]):
            # Setup the search space
            f_probs = probs[i, feat_index, :]
            f_range = range(
                int(max(x_i[feat_index] - eps, 0)),
                int(min(x_i[feat_index] + eps, 255) + 1),
            )

            # Look in the search space
            best_prob = -1
            best_idx = -1
            for idx in f_range:
                if f_probs[idx] > best_prob:
                    best_prob = f_probs[idx]
                    best_idx = idx

            # Update result
            x_i[feat_index] = best_idx

        # Update in batch
        x[i] = x_i

    # Convert to old dtype
    x = x / 255.0
    x = x.astype(ART_NUMPY_DTYPE).reshape(original_shape)

    # Clip to clip_values
    x = np.clip(x, clip_values[0], clip_values[1])

    return x, y

def _check_params(self) -> None:

    if not isinstance(self.eps, int) or self.eps < 0 or self.eps > 255:
        raise ValueError("The defense parameter must be between 0 and 255.")

    from art.estimators.classification.classifier import ClassifierMixin
    from art.estimators.estimator import NeuralNetworkMixin

    if hasattr(self, "pixel_cnn") and not (
        isinstance(self.pixel_cnn, ClassifierMixin) and isinstance(self.pixel_cnn, NeuralNetworkMixin)
    ):
        raise TypeError("PixelCNN model must be of type Classifier.")

    if np.array(self.clip_values[0] >= self.clip_values[1]).any():
        raise ValueError("Invalid `clip_values`: min >= max.")

    if self.clip_values[0] != 0:
        raise ValueError("`clip_values` min value must be 0.")

    if self.clip_values[1] != 1:
        raise ValueError("`clip_values` max value must be 1.")

    if self.batch_size <= 0:
        raise ValueError("The batch size `batch_size` has to be positive.")

    if not isinstance(self.verbose, bool):
        raise ValueError("The argument `verbose` has to be of type bool.")

In [23]:
# Normalization layer to normalize the input to expected input of model
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Add additional layer to normalize the input to expected input shape of model
norm_model = nn.Sequential(
            normalize,
            model
        ).to(device)
norm_model = norm_model.eval()

# Apply attack on the test dataloader
def test_adv(dataloader, activation, final_eval=False):
    # Reset the metrics
    # Normal model metrics
    mAP.reset()  
    if final_eval and not set_chlg_eval:
        mAPv.reset() 
        mAPt.reset() 
        mAPi.reset()
    
    # Adversarial Metrics
    mAP_adv.reset()  
    if final_eval and not set_chlg_eval:
        mAPv_adv.reset() 
        mAPt_adv.reset() 
        mAPi_adv.reset()

    
    print("DataLoader Started:")
    for batch, (img, (y1, y2, y3, y4)) in enumerate(dataloader):
        # Extract data in form of batches
        img, y1, y2, y3, y4 = img.cuda(), y1.cuda(), y2.cuda(), y3.cuda(), y4.cuda()
        
        
        #################### Normal Testing #####################
        # Predictions of the model on clean images
        tool, verb, target, triplet = norm_model(img)

        # Final Evaluation 
        if final_eval:
            # Get logits and CAMs for each individual category
            cam_i, logit_i = tool
            cam_v, logit_v = verb
            cam_t, logit_t = target

            # Update the mAP metrics for each prediction of clean images
            mAPi.update(y1.float().detach().cpu(), activation(logit_i).detach().cpu()) # Log metrics 
            mAPv.update(y2.float().detach().cpu(), activation(logit_v).detach().cpu()) # Log metrics 
            mAPt.update(y3.float().detach().cpu(), activation(logit_t).detach().cpu()) # Log metrics 
        mAP.update(y4.float().detach().cpu(), activation(triplet).detach().cpu()) # Log metrics 


        ############# Adversarial Attack Started ##############

        # Adversarial images obtained from FGSM
        adversarial_example = fast_gradient_sign_method(norm_model, img, (y1, y2, y3, y4), eps)

        # Save the clean and adversarial images
        # for i , img_adv in enumerate(adversarial_example):
        #     save_image(img[i], f'Images-Attack/Clean/img{i}-{batch}.png')
        #     save_image(img_adv, f'Images-Attack/Adv/img-adv{i}-{batch}.png')

        #### Finding Metrics for Image Quality ####

        adversarial_example , y4 = spatialSmoothing(adversarial_example.cpu() , y4.cpu())
        print(img)
        print(adversarial_example)

        print(k)

        # adversarial_example , y4 = adversarial_example.cuda() , y4.cuda()

        psnr_values_list.append(piq.psnr(adversarial_example, img).item())
        ssim_values_list.append(piq.ssim(adversarial_example, img).item())
        lpips_values_list.append(lpips(2*adversarial_example-1, 2*img-1).item())

        ######################## Adversarial Testing ########################
        
        # Testing the attacked images by the model
        tool_adv, verb_adv, target_adv, triplet_adv = norm_model(adversarial_example)
        
        if final_eval:
            # Update the mAP metrics for each prediction of adversarial images using logits
            
            cam_i, logit_i_adv = tool_adv
            cam_v, logit_v_adv = verb_adv
            cam_t, logit_t_adv = target_adv
            mAPi_adv.update(y1.float().detach().cpu(), activation(logit_i_adv).detach().cpu()) # Log metrics 
            mAPv_adv.update(y2.float().detach().cpu(), activation(logit_v_adv).detach().cpu()) # Log metrics 
            mAPt_adv.update(y3.float().detach().cpu(), activation(logit_t_adv).detach().cpu()) # Log metrics 
        mAP_adv.update(y4.float().detach().cpu(), activation(triplet_adv).detach().cpu()) # Log metrics 

    # End the update of each metric
    if final_eval:
        mAPv.video_end()
        mAPt.video_end()
        mAPi.video_end()
        
    if final_eval:
        mAPv_adv.video_end()
        mAPt_adv.video_end()
        mAPi_adv.video_end()



# Main function for applying attack

In [24]:
################### Perform FGSM attack ##################

# Set FGSM parameters
eps = 8/255
exp_number = 1

# Logs Saving File
version_adv = 'FGSM-Attack-SS'
logfile  = f'weights/Logs/{version_adv}.log'

#%% log config
header1 = "** Experiment for FGSM Attack **"
header2 = "** Eps: {}/255 \t Exp Number: {} **".format(int(eps * 255) , exp_number)
# header3 = "** LR Config: Init: {} | Peak: {} | Warmup Epoch: {} | Rise: {} | Decay {} | train params {} | all params {} **".format([float(f'{sch.get_last_lr()[0]:.6f}') for sch in lr_schedulers], [float(f'{v:.6f}') for v in wp_lr], warmups, power, decay_rate, pytorch_train_params, pytorch_total_params)
maxlen  = len(header1)
# header1 = "{}{}{}".format('*'*((maxlen-len(header1))//2+1), header1, '*'*((maxlen-len(header1))//2+1) )
# header2 = "{}{}{}".format('*'*((maxlen-len(header2))//2+1), header2, '*'*((maxlen-len(header2))//2+1) )
# header3 = "{}{}{}".format('*'*((maxlen-len(header3))//2+1), header3, '*'*((maxlen-len(header3))//2+1) )
# maxlen  = max(len(header1), len(header2), len(header3))
print("\n\n\n{}\n{}\n{}\n{}\n\n".format("*"*maxlen, header1, header2, "*"*maxlen), file=open(logfile, 'a+'))
print("Experiment started ...\n   logging outputs to: ", logfile)

# Reset the metrics globally
mAP.reset_global()
mAP_adv.reset_global()

mAPi.reset_global()
mAPv.reset_global()
mAPt.reset_global()

mAPi_adv.reset_global()
mAPv_adv.reset_global()
mAPt_adv.reset_global()

# Create a list to store image quality metrics for each batch
psnr_values_list = []
ssim_values_list = []
lpips_values_list = []
lpips = piq.LPIPS()

# Load each dataloader iteratively
for test_dataloader in test_dataloaders:
    # Pass the loader to the testing function
    test_adv(test_dataloader, activation, final_eval=True)
    
    print("Done...!. Printing Stats.")

# Compute the final update after each dataloader

####### Normal Metrics #####

if set_chlg_eval:
    mAP_i = mAP.compute_video_AP('i', ignore_null=set_chlg_eval)
    mAP_v = mAP.compute_video_AP('v', ignore_null=set_chlg_eval)
    mAP_t = mAP.compute_video_AP('t', ignore_null=set_chlg_eval)
else:
    mAP_i = mAPi.compute_video_AP(ignore_null=set_chlg_eval)
    mAP_v = mAPv.compute_video_AP(ignore_null=set_chlg_eval)
    mAP_t = mAPt.compute_video_AP(ignore_null=set_chlg_eval)

mAP_iv = mAP.compute_video_AP('iv', ignore_null=set_chlg_eval)
mAP_it = mAP.compute_video_AP('it', ignore_null=set_chlg_eval)
mAP_ivt = mAP.compute_video_AP('ivt', ignore_null=set_chlg_eval) 

####### Adversarial Metrics #####

if set_chlg_eval:
    mAP_i_adv = mAP_adv.compute_video_AP('i', ignore_null=set_chlg_eval)
    mAP_v_adv = mAP_adv.compute_video_AP('v', ignore_null=set_chlg_eval)
    mAP_t_adv = mAP_adv.compute_video_AP('t', ignore_null=set_chlg_eval)
else:
    mAP_i_adv = mAPi_adv.compute_video_AP(ignore_null=set_chlg_eval)
    mAP_v_adv = mAPv_adv.compute_video_AP(ignore_null=set_chlg_eval)
    mAP_t_adv = mAPt_adv.compute_video_AP(ignore_null=set_chlg_eval)

mAP_iv_adv = mAP_adv.compute_video_AP('iv', ignore_null=set_chlg_eval)
mAP_it_adv = mAP_adv.compute_video_AP('it', ignore_null=set_chlg_eval)
mAP_ivt_adv = mAP_adv.compute_video_AP('ivt', ignore_null=set_chlg_eval) 

# Print the results into the logging file

######### Printing Baseline Model Stats #########

print('-'*50, file=open(logfile, 'a+'))
print('Test Results\nPer-category AP: ', file=open(logfile, 'a+'))
print('-'*50, file=open(logfile, 'a+'))
print(f'Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT ', file=open(logfile, 'a+'))
print(f':::::: : {mAP_i["mAP"]:.4f} | {mAP_v["mAP"]:.4f} | {mAP_t["mAP"]:.4f} | {mAP_iv["mAP"]:.4f} | {mAP_it["mAP"]:.4f} | {mAP_ivt["mAP"]:.4f} ', file=open(logfile, 'a+'))
print('='*50, file=open(logfile, 'a+'))
print("Test results saved @ ", logfile)

######### Printing Adversarial Stats #########

print('-'*50, file=open(logfile, 'a+'))
print('Adversarial Test Results\nPer-category AP: ', file=open(logfile, 'a+'))
print('-'*50, file=open(logfile, 'a+'))
print(f'Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT ', file=open(logfile, 'a+'))
print(f':::::: : {mAP_i_adv["mAP"]:.4f} | {mAP_v_adv["mAP"]:.4f} | {mAP_t_adv["mAP"]:.4f} | {mAP_iv_adv["mAP"]:.4f} | {mAP_it_adv["mAP"]:.4f} | {mAP_ivt_adv["mAP"]:.4f} ', file=open(logfile, 'a+'))
print('='*50, file=open(logfile, 'a+'))
print('-'*50, file=open(logfile, 'a+'))
print(f'PSNR: {np.mean(psnr_values_list):.4f}    | SSIM: {np.mean(ssim_values_list):.4f}     | LPIPS: {1 - np.mean(lpips_values_list):.4f}', file=open(logfile, 'a+'))
print('-'*50, file=open(logfile, 'a+'))
print("Test results saved @ ", logfile)



Experiment started ...
   logging outputs to:  weights/Logs/FGSM-Attack-SS.log
DataLoader Started:
Filter size:  [1, 3, 3, 1]
tensor([[[[0.0471, 0.0471, 0.0471,  ..., 0.0549, 0.0549, 0.0549],
          [0.0471, 0.0471, 0.0471,  ..., 0.0549, 0.0549, 0.0549],
          [0.0471, 0.0471, 0.0471,  ..., 0.0549, 0.0549, 0.0549],
          ...,
          [0.0471, 0.0471, 0.0471,  ..., 0.0549, 0.0549, 0.0549],
          [0.0549, 0.0549, 0.0549,  ..., 0.0549, 0.0549, 0.0549],
          [0.0549, 0.0549, 0.0549,  ..., 0.0549, 0.0549, 0.0549]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000,

NameError: name 'k' is not defined

In [11]:
eps_list = [4,8,16,32]

In [12]:
################### Perform FGSM attack using the hyper params ##################
for eps in eps_list:
    # Set FGSM parameters
    eps = eps/255
    exp_number = 1
    
    # Logs Saving File
    version_adv = 'FGSM-Attack'
    logfile  = f'weights/LogsTooba/{version_adv}.log'
    
    #%% log config
    header1 = "** Experiment for FGSM Attack **"
    header2 = "** Eps: {}/255 **".format(int(eps * 255) )
    maxlen  = len(header1)

    print("\n\n\n{}\n{}\n{}\n{}\n\n".format("*"*maxlen, header1, header2, "*"*maxlen), file=open(logfile, 'a+'))
    print("Experiment started ...\n   logging outputs to: ", logfile)
    
    mAP.reset_global()
    mAP_adv.reset_global()
    
    mAPi.reset_global()
    mAPv.reset_global()
    mAPt.reset_global()
    
    mAPi_adv.reset_global()
    mAPv_adv.reset_global()
    mAPt_adv.reset_global()

    psnr_values_list = []
    ssim_values_list = []
    lpips_values_list = []
    lpips = piq.LPIPS()
    
    for test_dataloader in test_dataloaders:
        # index = 1
        test_adv(test_dataloader, model, activation, final_eval=True)
        
        # break
        ###### Normal ######
        print("Done...!. Printing Stats.")
    if set_chlg_eval:
        mAP_i = mAP.compute_video_AP('i', ignore_null=set_chlg_eval)
        mAP_v = mAP.compute_video_AP('v', ignore_null=set_chlg_eval)
        mAP_t = mAP.compute_video_AP('t', ignore_null=set_chlg_eval)
    else:
        mAP_i = mAPi.compute_video_AP(ignore_null=set_chlg_eval)
        mAP_v = mAPv.compute_video_AP(ignore_null=set_chlg_eval)
        mAP_t = mAPt.compute_video_AP(ignore_null=set_chlg_eval)
    
    mAP_iv = mAP.compute_video_AP('iv', ignore_null=set_chlg_eval)
    mAP_it = mAP.compute_video_AP('it', ignore_null=set_chlg_eval)
    mAP_ivt = mAP.compute_video_AP('ivt', ignore_null=set_chlg_eval) 
    
    ####### Adversarial Metrics #####
    
    if set_chlg_eval:
        mAP_i_adv = mAP_adv.compute_video_AP('i', ignore_null=set_chlg_eval)
        mAP_v_adv = mAP_adv.compute_video_AP('v', ignore_null=set_chlg_eval)
        mAP_t_adv = mAP_adv.compute_video_AP('t', ignore_null=set_chlg_eval)
    else:
        mAP_i_adv = mAPi_adv.compute_video_AP(ignore_null=set_chlg_eval)
        mAP_v_adv = mAPv_adv.compute_video_AP(ignore_null=set_chlg_eval)
        mAP_t_adv = mAPt_adv.compute_video_AP(ignore_null=set_chlg_eval)
    
    mAP_iv_adv = mAP_adv.compute_video_AP('iv', ignore_null=set_chlg_eval)
    mAP_it_adv = mAP_adv.compute_video_AP('it', ignore_null=set_chlg_eval)
    mAP_ivt_adv = mAP_adv.compute_video_AP('ivt', ignore_null=set_chlg_eval) 
    
    print('-'*50, file=open(logfile, 'a+'))
    print('Test Results: ', file=open(logfile, 'a+'))
    print('-'*50, file=open(logfile, 'a+'))
    print(f'Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT ', file=open(logfile, 'a+'))
    print(f':::::: : {mAP_i["mAP"]:.4f} | {mAP_v["mAP"]:.4f} | {mAP_t["mAP"]:.4f} | {mAP_iv["mAP"]:.4f} | {mAP_it["mAP"]:.4f} | {mAP_ivt["mAP"]:.4f} ', file=open(logfile, 'a+'))
    print('='*50, file=open(logfile, 'a+'))
    print("Test results saved @ ", logfile)
    
    
    print('-'*50, file=open(logfile, 'a+'))
    print('Adversarial Test Results: ', file=open(logfile, 'a+'))
    print('-'*50, file=open(logfile, 'a+'))
    print(f'Mean AP:  I  |  V  |  T  |  IV  |  IT  |  IVT ', file=open(logfile, 'a+'))
    print(f':::::: : {mAP_i_adv["mAP"]:.4f} | {mAP_v_adv["mAP"]:.4f} | {mAP_t_adv["mAP"]:.4f} | {mAP_iv_adv["mAP"]:.4f} | {mAP_it_adv["mAP"]:.4f} | {mAP_ivt_adv["mAP"]:.4f} ', file=open(logfile, 'a+'))
    print('='*50, file=open(logfile, 'a+'))
    print('-'*50, file=open(logfile, 'a+'))
    print(f'PSNR: {np.mean(psnr_values_list):.4f}    | SSIM: {np.mean(ssim_values_list):.4f}     | LPIPS: {1 - np.mean(lpips_values_list):.4f}', file=open(logfile, 'a+'))
    print('-'*50, file=open(logfile, 'a+'))
    print("Test results saved @ ", logfile)


Experiment started ...
   logging outputs to:  weights/LogsTooba/FGSM-Attack.log
DataLoader Started:
Done...!. Printing Stats.
DataLoader Started:
Done...!. Printing Stats.
DataLoader Started:
Done...!. Printing Stats.
DataLoader Started:
Done...!. Printing Stats.
DataLoader Started:
Done...!. Printing Stats.
DataLoader Started:
Done...!. Printing Stats.
DataLoader Started:
Done...!. Printing Stats.
DataLoader Started:
Done...!. Printing Stats.
DataLoader Started:
Done...!. Printing Stats.
Test results saved @  weights/LogsTooba/FGSM-Attack.log
Test results saved @  weights/LogsTooba/FGSM-Attack.log
